In [25]:
import xml.etree.ElementTree as ET
import numpy as np
import json

In [172]:
class Data():
    def __init__(self, P, numInst):
        root_node = ET.parse('Datas/InstanceP='+str(P)+'/Instance'+str(numInst)+'.xml').getroot()
        nb_pair=0
        nb_donor=0
        id_max=0
        
        #Calcul du nombre de pair, du nombre de donneur et de id_max
        for donor in root_node.findall('entry'):
            nb_donor=nb_donor+1
            id=donor.attrib['donor_id']
            if(int(id) > int(id_max)):
                id_max=donor.attrib['donor_id']
            if(donor.find('altruistic')==None):
                nb_pair=nb_pair+1
        print('nombre de pair : ',  nb_pair, ' ; nombre de donneur : ', nb_donor)
        print('id max : ', id_max)
        print()
        #Autre méthode utilisant le fichier json (ne fonctionne puisque le nombre de donneurn'est pas égale à nombre de patients x 
            # x110%)
        #with open('Datas/InstanceP='+str(50)+'/config.json') as json_data:
            #data_dict = json.load(json_data)
            #nb_paire=data_dict['patientsPerInstance']
            #nb_donor= int(nb_paire*1.12)
        
        #initialisation tableau des couts des échanges, tableau des donneurs altruistes et des paires
        cost = np.zeros((int(id_max)+1,int(id_max)+1))
        altruist = np.zeros(int(id_max)+1,int(id_max)+1)
        pair = np.zeros(int(id_max)+1,int(id_max)+1)
        
        #parcous du fichier html et remplissage des tableaux
        for donor in root_node.findall('entry'):
            #### Get the value from the attribute 'donor_id'###
            id = donor.attrib['donor_id']
            print('id du donneur :',int(id))
            for exchange in donor.findall('matches/match'):
                patient=exchange.find('recipient')
                score=exchange.find('score')
                if(patient.text!=None):
                    cost[int(id)][int(patient.text)]=int(score.text)
                #print('patient : '+ patient.text +' avec un score de ' + score.text)
                if(donor.find('altruistic')!=None):
                    altruist[int(id),int(patient)]=1
                else : 
                    pair[int(id),int(patient)]=1
                #print('le donneur est altruiste : ', altruist)
        self.nb_pair=nb_pair
        self.nb_donor=nb_donor
        self.cost=cost
        self.altruist=altruist
        self.pair=pair